In [206]:
import pandas as pd
import numpy as np
from pathlib import Path

In [207]:
df = pd.read_csv('../data/raw/sales_2023_2025_11.csv', header=None)

# Data Overview & Initial Assessment
The data was exported directly from Google Sheets and reflects manual data entry, resulting in multiple inconsistencies and missing values

In [208]:
# Display basic information about the dataframe
df.head()

,0,1,2,3,4,5,6,7
0,5076.0,1/5/2023,1/21/2022,Систем HP omen i5 7300hq 16 500ssd gtx 1080 8,Игорь,18550.0,23000.0,4450
1,6070.0,1/5/2023,12/21/2022,HP i7 8750 16 1256 1070,Паша,28150.0,33500.0,5350
2,6086.0,1/5/2023,12/24/2022,dell 13 i3 5005u 8 ssd128,Паша,4460.0,6500.0,2040
3,NaN,NaN,NaN,док станция + hdmi,NaN,NaN,700.0,700
4,5725.0,1/5/2023,9/9/2022,Монітор Samsung S24R350F,Игорь,4000.0,6000.0,2000


In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3430 entries, 0 to 3429
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2275 non-null   float64
 1   1       2274 non-null   object 
 2   2       2274 non-null   object 
 3   3       2544 non-null   object 
 4   4       1961 non-null   object 
 5   5       2397 non-null   float64
 6   6       2503 non-null   float64
 7   7       3430 non-null   int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 214.5+ KB


At this stage, the dataset does not contain proper column headers.  
When loading the CSV file, the first row was incorrectly interpreted as column names by pandas.

In [210]:
# Add column names
df.columns = ["sku",
    "sale_date",
    "purchase_date",
    "product_description",
    "manager",
    "purchase_price_uah",
    "sale_price_uah",
    "margin_uah"]

In [211]:
# Check for missing values
df.isna().sum()

sku                    1155
sale_date              1156
purchase_date          1156
product_description     886
manager                1469
purchase_price_uah     1033
sale_price_uah          927
margin_uah                0
dtype: int64

In [212]:
#check rows with key values missing
empty_rows = df[
    df["product_description"].isna() &
    df["sale_date"].isna() &
    df["purchase_price_uah"].isna() &
    df["sale_price_uah"].isna()
]

empty_rows.shape


(884, 8)

In [213]:
empty_rows.head()

,sku,sale_date,purchase_date,product_description,manager,purchase_price_uah,sale_price_uah,margin_uah
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### Structural Empty Rows Identification

Using key business fields (`product_description`, `sale_date`, `purchase_price_uah`, `sale_price_uah`),  
884 rows were identified as invalid sales records.

These rows contain values only in the `margin_uah` column (set to 0) and have missing values across all essential sales attributes.

In [214]:
#check rows with partial data
partial_rows = df[
    df.notna().sum(axis=1).between(2, 6)
]

partial_rows.shape


(276, 8)

In [215]:
partial_rows.head(10)

,sku,sale_date,purchase_date,product_description,manager,purchase_price_uah,sale_price_uah,margin_uah
3,NaN,NaN,NaN,док станция + hdmi,NaN,NaN,700.0,700
15,NaN,NaN,NaN,клава + мышка,NaN,NaN,550.0,550
31,NaN,NaN,NaN,возврат за обогреватель Розетка,NaN,NaN,3495.0,3495
35,NaN,NaN,NaN,lenovo thinkpad x280 i5 8350u 16 ssd256,коля,8000.0,10500.0,2500
51,NaN,NaN,NaN,lenovo x280 i5 8пок. 8gb / 16 gb 13шт,Коля,8000.0,10100.0,2100
54,NaN,NaN,NaN,thp S1 13'',NaN,NaN,5900.0,5900
56,NaN,NaN,NaN,HP a10 9600 6 ssd128 r7,NaN,NaN,8300.0,8300
64,NaN,NaN,NaN,lenovo x280 i5 8пок. 8gb / 16 gb 12 шт,Коля,42500.0,50500.0,8000
81,NaN,NaN,NaN,сервис + ssd128,NaN,NaN,800.0,800
85,NaN,NaN,NaN,lenovo x280 i5 8пок. 8gb / 16 gb 7шт,NaN,42500.0,50500.0,8000


### Partially Filled Rows Analysis

In addition to fully empty structural rows, 276 partially filled rows were identified.

These rows fall into two distinct categories:

**1. Non-sales operational records**  
Rows representing accessories, refunds, or miscellaneous transactions (e.g. keyboards, docks, refunds).  

**2. Incomplete sales records**  
Rows that represent actual laptop sales but have missing key attributes such as sale or purchase dates.  



## Data Quality Issues

 - Structural empty rows

 - 884 rows

 - artifacts of batch-based data extraction

 - Non-sales operational records

 - accessories, refunds, miscellaneous items

 - mixed with sales data

 - Incomplete sales records

 - missing dates and/or identifiers

 - Missing identifiers (sale_id)

 - not consistently populated

 - Dates stored as strings

 - require parsing and standardization

 - Free-text product descriptions

 - inconsistent naming and formatting

-------

## Stage 2 — Data Cleaning

### Step 1 — Remove structural empty rows



In [216]:
# define key business fields required for a valid sale
key_fields = [
    "product_description",
    "sale_date",
    "purchase_price_uah",
    "sale_price_uah"
]

# number of rows before cleaning
rows_before = df.shape[0]

# remove structural empty rows
structural_empty_mask = df[key_fields].isna().all(axis=1)
df = df[~structural_empty_mask].copy()

# number of rows after cleaning
rows_after = df.shape[0]

print("Removed:", structural_empty_mask.sum())



Removed: 884


### Step 2 — Separate non-sales operational records

Rows without a purchase price are treated as non-sales operational records  
(accessories, refunds, miscellaneous transactions) and are separated into a
dedicated dataset for independent analysis.


In [217]:
# number of rows before separation
rows_before = df.shape[0]

# identify non-sales operational records (no purchase price)
non_sales_mask = (
    df["purchase_price_uah"].isna() &
    df["product_description"].notna()
)

# create separate dataframe for non-sales records
other_df = df[non_sales_mask].copy()

# keep only sales records in main dataframe
df = df[~non_sales_mask].copy()

rows_before, df.shape[0], other_df.shape[0]


(2546, 2397, 149)

### Step 3 — Handle incomplete sales records

Sales records with missing critical attributes are retained in the dataset.  
Missing values are explicitly flagged to preserve data integrity and enable
transparent downstream handling.


In [218]:
# create flags for missing critical fields
df["missing_sale_date"] = df["sale_date"].isna()
df["missing_purchase_date"] = df["purchase_date"].isna()
df["missing_sku"] = df["sku"].isna()

# quick overview of incomplete records
df[[
    "missing_sale_date",
    "missing_purchase_date",
    "missing_sku"
]].sum()


missing_sale_date        124
missing_purchase_date    124
missing_sku              123
dtype: int64

### Step 4 — Parse and standardize date fields

Date columns are converted from string format to datetime.
Invalid or missing values are preserved as NaT to avoid introducing fabricated data.


In [219]:
# parse date columns to datetime (keep invalid/missing as NaT)
date_cols = ["sale_date", "purchase_date"]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# quick validation
df[date_cols].dtypes


sale_date        datetime64[ns]
purchase_date    datetime64[ns]
dtype: object

In [220]:
# count missing dates after parsing
df[date_cols].isna().sum()


sale_date        124
purchase_date    124
dtype: int64

### Step 5 — Standardize manager field

Missing manager values are replaced with a standardized placeholder to ensure
consistent categorical analysis.


In [221]:
# Count occurrences of each manager to make mapping 
df['manager']. value_counts()

manager
Коля     1154
Паша      642
Игорь     135
-          17
Артем       6
 Коля       2
коля        1
Пашка       1
Паша        1
Name: count, dtype: int64

In [222]:
# Map manager names to standardized identifiers
manager_map = {
    "Игорь": "manager_1",
    "Паша": "manager_2",
    "Пашка": "manager_2",
    "Паша " : "manager_2",
    "Коля": "manager_3",
    "коля": "manager_3",
    " Коля": "manager_3",
    "Артем": "manager_4",
    "-": "unknown_manager",
    "": "unknown_manager"
}

df["manager"] = df["manager"].replace(manager_map)
df["manager"] = df["manager"].fillna("unknown_manager")
df["manager"] = df["manager"].astype("category")


In [223]:
# fixed yearly USD exchange rates (for analysis purposes)
USD_RATES_BY_YEAR = {
    2023: 38.5,
    2024: 40.5,
    2025: 41.5
}

# extract sale year
df["sale_year"] = df["sale_date"].dt.year

# map yearly exchange rate
df["usd_rate"] = df["sale_year"].map(USD_RATES_BY_YEAR)

# convert prices to USD
price_cols = ["purchase_price_uah", "sale_price_uah", "margin_uah"]

for col in price_cols:
    df[col.replace("_uah", "_usd")] = df[col] / df["usd_rate"]


### Step 6 — Normalize product description text

Product descriptions are lightly normalized to ensure consistent text format.
No semantic parsing or feature extraction is performed at this stage.


In [224]:
# basic text normalization for product descriptions
df["product_description"] = (
    df["product_description"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)


In [225]:
# convert SKU to string type
df["sku"] = df["sku"].astype("string")

In [226]:
# final overview of dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2397 entries, 0 to 3397
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   sku                    2274 non-null   string        
 1   sale_date              2273 non-null   datetime64[ns]
 2   purchase_date          2273 non-null   datetime64[ns]
 3   product_description    2395 non-null   object        
 4   manager                2397 non-null   category      
 5   purchase_price_uah     2397 non-null   float64       
 6   sale_price_uah         2393 non-null   float64       
 7   margin_uah             2397 non-null   int64         
 8   missing_sale_date      2397 non-null   bool          
 9   missing_purchase_date  2397 non-null   bool          
 10  missing_sku            2397 non-null   bool          
 11  sale_year              2273 non-null   float64       
 12  usd_rate               2273 non-null   float64       
 13  purchase

In [227]:
df.head()

,sku,sale_date,purchase_date,product_description,manager,purchase_price_uah,sale_price_uah,margin_uah,missing_sale_date,missing_purchase_date,missing_sku,sale_year,usd_rate,purchase_price_usd,sale_price_usd,margin_usd
0,5076.0,2023-01-05,2022-01-21,систем hp omen i5 7300hq 16 500ssd gtx 1080 8,manager_1,18550.0,23000.0,4450,False,False,False,2023.0,38.5,481.818182,597.402597,115.584416
1,6070.0,2023-01-05,2022-12-21,hp i7 8750 16 1256 1070,manager_2,28150.0,33500.0,5350,False,False,False,2023.0,38.5,731.168831,870.129870,138.961039
2,6086.0,2023-01-05,2022-12-24,dell 13 i3 5005u 8 ssd128,manager_2,4460.0,6500.0,2040,False,False,False,2023.0,38.5,115.844156,168.831169,52.987013
4,5725.0,2023-01-05,2022-09-09,монітор samsung s24r350f,manager_1,4000.0,6000.0,2000,False,False,False,2023.0,38.5,103.896104,155.844156,51.948052
5,6085.0,2023-01-05,2022-12-24,dell 13 i3 5005u 8 ssd128,manager_2,4460.0,6500.0,2040,False,False,False,2023.0,38.5,115.844156,168.831169,52.987013


In [228]:
other_df.head()

,sku,sale_date,purchase_date,product_description,manager,purchase_price_uah,sale_price_uah,margin_uah
3,NaN,NaN,NaN,док станция + hdmi,NaN,NaN,700.0,700
15,NaN,NaN,NaN,клава + мышка,NaN,NaN,550.0,550
31,NaN,NaN,NaN,возврат за обогреватель Розетка,NaN,NaN,3495.0,3495
54,NaN,NaN,NaN,thp S1 13'',NaN,NaN,5900.0,5900
56,NaN,NaN,NaN,HP a10 9600 6 ssd128 r7,NaN,NaN,8300.0,8300


### Step 7 — Save cleaned datasets

The cleaned sales dataset and separated operational records are saved for
downstream analysis and feature engineering.


In [229]:
df.to_csv("../data/processed/sales_clean.csv", index=False)
other_df.to_csv("../data/processed/other_operations.csv", index=False)

## Final Summary

- Structural empty rows and non-sales operational records were identified and separated.
- Real sales records were retained, with missing critical fields explicitly flagged.
- Date fields were standardized without fabricating values.
- Manager field and product descriptions were normalized for consistency.
- Currency-converted features were added while preserving original values.
- Clean, analysis-ready datasets were saved for further feature engineering.
